In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    api_key="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",
    max_retries=2,
    # other params...
)

In [22]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://internshala.com/internship/detail/devops-internship-in-bangalore-at-sigmoid1749452837/")
docs=loader.load().pop().page_content
docs

"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\nDevOps Internship in Bangalore at Sigmoid\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nClose\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCancel\nClose\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nClose\nClose\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nClose\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCancel\nYes\n\n\n\n\n\n\n\n\n\n\n\n\n        Oops, your browser does not support Javascript. Please use Internshala in another browser.\n    \n\n\n\n\n            Loading, please wait...\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                        Student\n                                    \n\n                                        Employer / T&P\n                                    \n\n\n\n\n\n\n\n\n\n                                    Login with Google\n                                \n\n\n\n\nOR\n\n\n\n\n\n\nEmail\n\n\n\nPassword\n\n\n\nForgot

In [3]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


In [4]:
class JD(BaseModel):
    role:str=Field(description="Role of the job")
    experience:int=Field(description="experience needed for the job in years")
    skills:list[str]=Field(description="skills required of the job")
parser =JsonOutputParser(pydantic_object=JD)


In [5]:
prompt = """
You are an intelligent extraction agent.

 {format}



Here is the job description:
{job_description}

Only return a valid JSON response. Do not include any explanation or extra text.
"""


In [6]:
prompt_temp=PromptTemplate.from_template(template=prompt,nput_variables=["job_description"],partial_variables={"format":parser.get_format_instructions()})


In [7]:
chain=prompt_temp|llm|parser

In [8]:
json_res=chain.invoke(input={"job_description":docs},)



In [9]:
import pandas as pd

In [10]:
df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [11]:
import chromadb
from chromadb.config import Settings
from chromadb import PersistentClient

In [12]:
client = PersistentClient("my_chroma_db")
collection = client.get_or_create_collection("my_collection")

In [13]:
import uuid


if not collection.count():
    for _,row in df.iterrows():
        print(row)
        collection.add(documents=row["Techstack"],metadatas={"links":row["Links"]},ids=[str(uuid.uuid4())])

In [14]:
json_res

{'role': 'DevOps Intern',
 'experience': 0,
 'skills': ['Python',
  'DevOps tools',
  'Automation systems',
  'Cloud computing',
  'Big data tools']}

In [15]:
links=collection.query(query_texts=json_res["skills"],n_results=2).get("metadatas")
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}]]

In [16]:
prompt_email = PromptTemplate.from_template(
        template=    """
            ### JOB DESCRIPTION:
            {job_description}

            ### INSTRUCTION:
            You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
            the seamless integration of business processes through automated tools. 
            Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
            process optimization, cost reduction, and heightened overall efficiency. 
            Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
            in fulfilling their needs.
            Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
            Remember you are Mohan, BDE at AtliQ. 
            Do not provide a preamble.
            ### EMAIL (NO PREAMBLE):

            """,

        )


In [17]:
email_chain=prompt_email|llm

In [20]:
from markdown import markdown
from IPython.display import display, HTML

In [21]:
res=email_chain.invoke({
    "job_description": docs,
    "link_list": links
})
html=markdown(res.content)
display(HTML(html))